In [ ]:
#!pip install beautifulsoup4 requests selenium

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re

Collecting Signal IDs

In [ ]:
# The base URL of the website we want to crawl
base_url = 'https://www.mql5.com/en/signals/mt5/list/page'

# The list of pages we want to crawl // Useless code 
page_urls = ['1']

# The CSV file we want to write the data to
csv_file = open('signal_list.csv', 'w', newline='')
writer = csv.writer(csv_file)
writer.writerow(['Signal Id'])

# Loop over each page URL and extract the Signal ID data
for page_url in page_urls:
    # Construct the full URL for the page
    url = base_url + page_url

    # Make a GET request to the page
    response = requests.get(url)

    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the element has id 
    ids = soup.find_all('div', class_='button button_white-and-green')

    #extrac the id
    for id in ids:
        name = id.get("data-id")
        # Write the data to the CSV file
        writer.writerow([name])

# Close the CSV file
csv_file.close()


Viewing IDs List

In [3]:
IDs = pd.read_csv("signal_list.csv")
IDs.head(10)


<bound method NDFrame.head of     Signal Id
0     1368342
1     1636506
2      843451
3     1486477
4     1818780
5     1598419
6     1245170
7     1722050
8     1725375
9     1334849
10    1712504
11    1857419
12    1788678
13    1766488
14    1726461
15    1829142
16    1895985
17     951062
18    1778541
19    1888377
20    1405177
21    1759554
22    1694912
23    1184516
24    1855883
25    1273204
26    1894325
27    1780040
28    1531887
29     910374
30    1526229
31    1312512
32    1612455
33    1274869
34    1594934
35    1908576
36    1507561
37    1607553
38    1415131
39    1882495
40    1059619
41    1196985
42    1500355
43    1722761
44    1279728
45    1729929
46    1833703
47    1815355>

In [4]:
base_url_singleID = 'https://www.mql5.com/en/signals/'

#load the csv containing Signal IDs
with open('signal_details.csv', 'w', newline='') as signal_detailsFile:
  writer = csv.writer(signal_detailsFile)
  writer.writerow(['Signal Id','Trades','ProfitTrades','LossTrades','BestTrades','WorstTrades','GrossProfit','GrossLoss','SharpeRatio','RecoveryFactor','ProfitFactor','ExpectedPayoff'])
  for row in IDs['Signal Id']:
    url = base_url_singleID + str(row) + "?source=Unknown#!tab=stats"
    
    #Crawl each signal
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    values = soup.find_all('div', class_='s-data-columns__value')

    # Loop over each product and extract the data
    signalTrades = values[0].text 


    # Get total number of profit trades
    s = values[1].text
    number_regex = r"([\d\s]+)\s*\(.*\)"
    match = re.search(number_regex, s)
    if match:
        number = match.group(1)
        # Remove spaces and normalize the number
        normalized_number = float(number.replace(" ", "").replace(",", ""))
    signalProfitTrades = normalized_number

    # Get total number of loss trades

    s = values[2].text
    number_regex = r"([\d\s]+)\s*\(.*\)"
    match = re.search(number_regex, s)
    if match:
        number = match.group(1)
        # Remove spaces and normalize the number
        normalized_number = float(number.replace(" ", "").replace(",", ""))
    signalLossTrades = normalized_number

    signalBestTrade = values[3].find('i').text
    signalWorstTrade = values[4].find('i').text

    signalGrossProfit = values[5].find('i').text
    signalGrossLoss = values[6].find('i').text

    signalSharpeRatio = values[9].text

    signalRecoveryFactor = values[15].text


    signalProfitFactor = values[18].text

    signalExpectedPayoff = values[19].find('i').text
    writer.writerow([row,signalTrades,signalProfitTrades,signalLossTrades,signalBestTrade,signalWorstTrade,signalGrossProfit,signalGrossLoss,signalSharpeRatio,signalRecoveryFactor,signalProfitFactor,signalExpectedPayoff])
    




View Signal Information

In [5]:
signals = pd.read_csv("signal_details.csv")
signals.head(10)

,Signal Id,Trades,ProfitTrades,LossTrades,BestTrades,WorstTrades,GrossProfit,GrossLoss,SharpeRatio,RecoveryFactor,ProfitFactor,ExpectedPayoff
0,1368342,1 853,1310.0,543.0,807.28,-443.63,29 989.77,-14 138.19,0.16,12.61,2.12,8.55
1,1636506,203,156.0,47.0,133.49,-80.75,2 135.53,-428.41,0.41,20.01,4.98,8.41
2,843451,1 113,787.0,326.0,254.65,-276.76,8 006.82,-5 392.07,0.12,3.38,1.48,2.35
3,1486477,519,398.0,121.0,94.36,-260.41,5 143.11,-2 723.36,0.30,4.74,1.89,4.66
4,1818780,364,231.0,133.0,1 768.00,-879.20,26 694.20,-11 491.85,0.14,11.28,2.32,41.76
5,1598419,469,300.0,169.0,1 793.00,-1 894.46,28 443.51,-16 389.95,0.12,4.14,1.74,25.70
6,1245170,3 061,1947.0,1114.0,16.48,-13.65,5 003.41,-2 740.77,0.19,15.65,1.83,0.74
7,1722050,387,269.0,118.0,1 759.00,-875.60,27 970.27,-11 483.92,0.14,12.27,2.44,42.60
8,1725375,1 425,1048.0,377.0,366.09,-257.81,8 784.34,-4 915.82,0.12,3.95,1.79,2.71
9,1334849,788,598.0,190.0,44.96,-57.80,1 692.97,-1 030.59,0.16,3.46,1.64,0.84
